In [1]:
!pip install "torch==2.4.0"
!pip install "peft==0.13.2"
!pip install "trl==0.11.1"
!pip install "transformers==4.44.2"
!pip install "bitsandbytes==0.44.1"
!pip install "accelerate==0.34.2"
!pip install "huggingface_hub==0.25.1"
!pip install "unsloth[cu121-ampere-torch240] @ git+https://github.com/unslothai/unsloth.git"     

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-9fuwckr3/unsloth_3ae1a49a31594a209aaac1b2f5e254ef
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-9fuwckr3/unsloth_3ae1a49a31594a209aaac1b2f5e254ef
  Resolved https://github.com/unslothai/unsloth.git to commit 3085f4c3daacc63939e78e3c87759d0d03c5a71f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached https://download.pytorch.org/whl/cu121/xformers-0.0.27.post2-cp311-cp311-manylinux2014_x86_64.whl (20.8 MB)


In [2]:
from unsloth import FastLanguageModel

max_seq_length = 3072
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='/workspace/outputs_new/checkpoint-91',
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.2: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.254 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`
Unsloth 2024.10.2 patched 64 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 5120)
        (layers): ModuleList(
          (0-63): 64 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=256, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=256, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lo

In [3]:
prompt = """
### Instruction:
Вы — опытный ментор по программированию, помогающий студентам учиться на своих ошибках. Ваша задача — давать студенту полезные советы и направлять его, не предоставляя готового решения задачи. Когда студент показывает код, вы должны:
Никогда не давать готовое решение задачи.
Указать на ошибку или недочет, если они есть.
Объяснить, в чем состоит проблема, и предложить направление для исправления.
При необходимости объяснять концепции, которые могут помочь студенту найти решение самостоятельно.
Объясните, что может быть не так в его решении, используя за основу одну или несколько из следующих фраз:
- Обратите внимание на неверный...
- ...необходимо проверить, что...
- Вы забыли поставить...
- Необходимо использовать...
- Вы некорректно...
- Проверьте написание...
- В данном случае не нужно...
- ...неверный синтаксис...
- Ваш код охватывает не все возможные случаи...
- Попробуйте дополнить...
- Вы использовали неверную...
- Ошибка при обращении к...
- Проверьте, что...
- Ваш код использует неверный...

### Input:
{}

### Response:
{}
"""

In [4]:
def get_hint(input_,):
    inputs = tokenizer(
    [
        prompt.format(
            input_,
            '',
        )
    ], return_tensors = 'pt').to('cuda')
    
    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
    return tokenizer.batch_decode(outputs)

In [5]:
import pandas as pd

In [6]:
test_solutions = pd.read_excel('/workspace/test/solutions.xlsx')
test_tasks = pd.read_excel('/workspace/test/tasks.xlsx')
test_tests = pd.read_excel('/workspace/test/tests.xlsx')

In [7]:
def combine_data(solutions_, tasks_, tests_):
    solutions, tasks, tests = solutions_.copy(), tasks_.copy(), tests_.copy()

    grouped_tests = tests.groupby('task_id').apply(
        lambda x: "; ".join([
            f"number: {row['number']}, type: {row['type']}, input: {row['input']}, output: {row['output']}"
            for _, row in x.iterrows()
        ])
    ).reset_index()
    grouped_tests = grouped_tests.rename(columns={0: 'tests'})

    merged_df = pd.merge(solutions, tasks, left_on='task_id', right_on='id', suffixes=('_solution', '_task'))
    final_df = pd.merge(merged_df, grouped_tests, how='left', on='task_id')

    return final_df

In [8]:
test = combine_data(test_solutions, test_tasks, test_tests)

/tmp/ipykernel_15924/1862003407.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_tests = tests.groupby('task_id').apply(


In [9]:
test_text = test[['description', 'author_solution', 'student_solution', 'tests']]

In [10]:
input_template = '''
Условие задачи: {description}

Эталонное решение: {author_solution}

Решение студента: {student_solution}
'''

output_template = '''
Подсказка: {author_comment}

'''

In [11]:
test_df = pd.DataFrame({
    'input': test_text.apply(lambda row: input_template.format(
        description=row['description'],
        author_solution=row['author_solution'],
        student_solution=row['student_solution']
    ), axis=1)
})

In [12]:
from tqdm import tqdm

In [13]:
hints = []
for i in tqdm(range(len(test_df))):
    hint = get_hint(test_df.iloc[i].input)
    hints.append(hint[0].split('Подсказка:')[-1].split('\n\n<|im_end|>')[0].strip()) # проверить по чему сплитить !!!

100%|██████████| 325/325 [25:52<00:00,  4.78s/it]


In [14]:
import pickle

with open('hints_synthlora_newparams_qwen32b_91steps.pkl', 'wb') as f:
    pickle.dump(hints, f)